In [123]:
import pandas as pd
import streamlit as st
import plotly.express as px
import numpy as np

# TO RUN THIS, USE TERMINAL
# streamlit run e:\iCloudDrive\Drop\Python_dropbox\Python\2024\Streamlit_Template_2.py
# Turn on 'Always re-run' option on your app (in the web browser), then every time you save code changes, they'll automatically show in the app

# File and folder path
fol = ''
fn = 'FS_Golf_DB.xlsx'
df = pd.read_excel(fol+fn)
fol2 = 'C:/Users/dmomb/OneDrive/Documents/Golf/'

# Cleaning the column names
# Clean column names to ensure consistency
df.columns = df.columns.str.replace(r'[^\w\s]', '', regex=True).str.replace('\xa0', ' ').str.strip().str.replace(' ', '_')

# Convert Time to strings
df['Session'] = df['Time'].dt.strftime('%Y %b %d %I:%M %p')  # Or any simpler string representation

# Ensure Session is treated as categorical
#df['Session'] = df['Session'].astype('category')

In [124]:
df.columns

Index(['Mombo_ShotID', 'Club', 'Time', 'Golfer', 'Shot', 'Video', 'Ball_mph',
       'Club_mph', 'Smash_Factor', 'Carry_yds', 'Total_yds', 'Roll_yds',
       'Swing_H', 'Spin_rpm', 'Height_ft', 'Time_s', 'AOA', 'Spin_Loft',
       'Swing_V', 'Spin_Axis', 'Lateral_yds', 'Shot_Type', 'FTP', 'FTT',
       'Dynamic_Loft', 'Club_Path', 'Launch_H', 'Launch_V', 'Low_Point_ftin',
       'DescentV', 'Curve_Dist_yds', 'Lateral_Impact_in', 'Vertical_Impact_in',
       'Mode', 'Location', 'Unnamed_35', 'Unnamed_36', 'Unnamed_37',
       'Unnamed_38', 'Unnamed_39', 'Unnamed_40', 'Comment', 'User1', 'User2',
       'Exclude', 'Session'],
      dtype='object')

In [125]:
# Convert value with error handling
def convert_value(value):
    """
    Converts any value like '40R' or '20L' to numerical form:
    - '40R' becomes -40
    - '20L' becomes 20
    Returns None for invalid entries.
    """
    try:
        value = str(value).strip()  # Ensure value is a string and remove spaces
        number, direction = value[:-1], value[-1].upper()  # Split number and direction
        number = float(number)
        return -number if direction == 'R' else number if direction == 'L' else None
    except (ValueError, AttributeError):
        return None  # Return None for invalid entries

# Convert entire column and handle missing column errors
def convert_column(df, col):
    """
    Converts all values in a column using convert_value.
    Handles missing column errors gracefully.
    """
    if col in df.columns:
        df[col] = df[col].apply(convert_value)
        print(f"Column '{col}' successfully converted.")
    else:
        st.error(f"The column '{col}' is missing from the data.")
# Convert all the columns that have L & R in the data        
lrcols = ['Swing_H','Spin_Axis','Lateral_yds','FTP','FTT','Club_Path','Launch_H']
for col in lrcols:
    convert_column(df,col)

Column 'Swing_H' successfully converted.
Column 'Spin_Axis' successfully converted.
Column 'Lateral_yds' successfully converted.
Column 'FTP' successfully converted.
Column 'FTT' successfully converted.
Column 'Club_Path' successfully converted.
Column 'Launch_H' successfully converted.


In [126]:
df

,Mombo_ShotID,Club,Time,Golfer,Shot,Video,Ball_mph,Club_mph,Smash_Factor,Carry_yds,...,Unnamed_36,Unnamed_37,Unnamed_38,Unnamed_39,Unnamed_40,Comment,User1,User2,Exclude,Session
0,1,Pitching Wedge,2024-10-31 20:23:00,Dave,1,NaN,93.6,76.5,1.22,120.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM
1,2,Pitching Wedge,2024-10-31 20:23:00,Dave,2,NaN,71.9,71.3,1.01,76.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM
2,3,Pitching Wedge,2024-10-31 20:23:00,Dave,3,NaN,79.7,66.5,1.20,93.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM
3,4,Pitching Wedge,2024-10-31 20:23:00,Dave,4,NaN,75.2,62.0,1.21,83.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM
4,5,Pitching Wedge,2024-10-31 20:23:00,Dave,5,NaN,71.6,68.6,1.04,78.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,631,Pitching Wedge,2024-12-21 17:21:00,Dave,20,NaN,89.6,67.5,1.33,114.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Dec 21 05:21 PM
631,632,Pitching Wedge,2024-12-21 17:21:00,Dave,21,NaN,80.1,68.2,1.17,96.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Dec 21 05:21 PM
632,633,Pitching Wedge,2024-12-21 17:21:00,Dave,23,NaN,80.9,67.8,1.19,95.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Dec 21 05:21 PM
633,634,Pitching Wedge,2024-12-21 17:21:00,Dave,24,NaN,93.6,73.8,1.27,124.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Dec 21 05:21 PM


In [127]:
#df.to_excel(fol2+"Golf_DB_transform.xlsx")

In [128]:
dimensions = ['Club','Golfer','Session','Shot_Type','Mode']

num_columns = ['Ball_mph', 'Club_mph', 'Smash_Factor', 'Carry_yds','Total_yds', 'Roll_yds', 'Swing_H', 'Height_ft', 'Time_s', 'AOA',
       'Spin_Loft', 'Swing_V', 'Spin_Axis', 'Lateral_yds', 'FTP', 'FTT','Dynamic_Loft', 'Club_Path', 'Launch_H', 'Launch_V', 'DescentV',
       'Curve_Dist_yds', 'Lateral_Impact_in', 'Vertical_Impact_in']

clubs = ['Driver','3 Wood','5 Wood','4 Iron','5 Iron','6 Iron','7 Iron', '8 Iron','9 Iron', 'Pitching Wedge',  'Gap Wedge','Sand Wedge' , 'Lob Wedge']

In [129]:
# 1. Parse & sort
df['Session_dt'] = pd.to_datetime(df['Session'], format='%Y %b %d %I:%M %p')
df = df.sort_values('Session_dt').copy()

In [130]:
df['Session_dt'].head(50)

0    2024-10-31 20:23:00
15   2024-10-31 20:23:00
14   2024-10-31 20:23:00
12   2024-10-31 20:23:00
11   2024-10-31 20:23:00
10   2024-10-31 20:23:00
9    2024-10-31 20:23:00
8    2024-10-31 20:23:00
13   2024-10-31 20:23:00
6    2024-10-31 20:23:00
5    2024-10-31 20:23:00
4    2024-10-31 20:23:00
3    2024-10-31 20:23:00
2    2024-10-31 20:23:00
1    2024-10-31 20:23:00
7    2024-10-31 20:23:00
29   2024-11-03 14:56:00
30   2024-11-03 14:56:00
31   2024-11-03 14:56:00
32   2024-11-03 14:56:00
36   2024-11-03 14:56:00
34   2024-11-03 14:56:00
35   2024-11-03 14:56:00
37   2024-11-03 14:56:00
28   2024-11-03 14:56:00
33   2024-11-03 14:56:00
27   2024-11-03 14:56:00
26   2024-11-03 14:56:00
25   2024-11-03 14:56:00
16   2024-11-03 14:56:00
18   2024-11-03 14:56:00
19   2024-11-03 14:56:00
20   2024-11-03 14:56:00
17   2024-11-03 14:56:00
22   2024-11-03 14:56:00
23   2024-11-03 14:56:00
24   2024-11-03 14:56:00
21   2024-11-03 14:56:00
62   2024-11-03 20:05:00
56   2024-11-03 20:05:00


In [131]:
# 1. Parse & sort
df['Session_dt'] = pd.to_datetime(df['Session'], format='%Y %b %d %I:%M %p')
df = df.sort_values('Session_dt').copy()

# 2. Convert Session to ordered categorical
sorted_sessions = df['Session'].unique()
df['Session'] = pd.Categorical(
    df['Session'],
    categories=sorted_sessions,
    ordered=True
)

# 3. Pivot
df_pivot = df.pivot_table(
    index='Club',
    columns=['Golfer', 'Session'],
    values=num_columns, 
    aggfunc='mean'
)
# 4. (Optional) Reindex columns to ensure Session is in ascending chronological order
df_pivot = df_pivot.reindex(columns=sorted_sessions, level='Session')

C:\Users\dmomb\AppData\Local\Temp\ipykernel_2300\3750318488.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_pivot = df.pivot_table(


In [117]:
df_pivot

AOA                                            \
Golfer                         Alex                                             
Session        2024 Nov 03 08:05 PM 2024 Nov 06 12:54 PM 2024 Nov 10 02:56 PM   
Club                                                                            
4 Iron                          NaN                  NaN                  NaN   
5 Iron                          NaN                  NaN                  NaN   
5 Wood                          NaN                  NaN             3.916667   
6 Iron                          NaN                  NaN             9.941667   
7 Iron                     4.266667                  NaN             9.758571   
8 Iron                          NaN                  NaN            10.294118   
9 Iron                          NaN            10.725000                  NaN   
Driver                          NaN                  NaN             2.433333   
Gap Wedge                       NaN                  NaN             9.210000   
Lob Wedge                       NaN             3.733333                  NaN   
Pitching Wedge                  NaN                  NaN                  NaN   
Sand Wedge                      NaN                  NaN                  NaN   

                                                                               \
Golfer                                                                          
Session        2024 Nov 29 07:05 PM 2024 Dec 08 03:19 PM 2024 Dec 08 04:34 PM   
Club                                                                            
4 Iron                     4.410526                  NaN                  NaN   
5 Iron                     4.936364                  NaN                 4.28   
5 Wood                          NaN                  NaN                  NaN   
6 Iron                     4.355556                  NaN                  NaN   
7 Iron                     6.966667                  NaN                  NaN   
8 Iron                          NaN                  NaN                  NaN   
9 Iron                     6.692308             7.271429                  NaN   
Driver                          NaN                  NaN                  NaN   
Gap Wedge                       NaN                  NaN                  NaN   
Lob Wedge                       NaN                  NaN                  NaN   
Pitching Wedge                  NaN                  NaN                  NaN   
Sand Wedge                 6.825000                  NaN                  NaN   

                                                                               \
Golfer                                              Dave                        
Session        2024 Dec 08 07:54 PM 2024 Oct 31 08:23 PM 2024 Nov 03 02:56 PM   
Club                                                                            
4 Iron                          NaN                  NaN                  NaN   
5 Iron                          NaN                  NaN                  NaN   
5 Wood                          NaN                  NaN                  NaN   
6 Iron                          NaN                  NaN                  NaN   
7 Iron                          NaN                  NaN             5.533333   
8 Iron                        6.264                  NaN                  NaN   
9 Iron                          NaN                  NaN                  NaN   
Driver                          NaN                  NaN                  NaN   
Gap Wedge                       NaN                  NaN                  NaN   
Lob Wedge                       NaN                  NaN                  NaN   
Pitching Wedge                  NaN                 6.75                  NaN   
Sand Wedge                      NaN                  NaN                  NaN   

                                     ...            Total_yds  \
Golfer                               ...                 Dave   
Session    

AOA                                            \
Golfer                         Alex                                             
Session        2024 Dec 08 03:19 PM 2024 Dec 08 04:34 PM 2024 Dec 08 07:54 PM   
Club                                                                            
4 Iron                          NaN                  NaN                  NaN   
5 Iron                          NaN                 4.28                  NaN   
5 Wood                          NaN                  NaN                  NaN   
6 Iron                          NaN                  NaN                  NaN   
7 Iron                          NaN                  NaN                  NaN   
8 Iron                          NaN                  NaN                6.264   
9 Iron                     7.271429                  NaN                  NaN   
Driver                          NaN                  NaN                  NaN   
Gap Wedge                       NaN                  NaN                  NaN   
Lob Wedge                       NaN                  NaN                  NaN   
Pitching Wedge                  NaN                  NaN                  NaN   
Sand Wedge                      NaN                  NaN                  NaN   

                                                                               \
Golfer                                                                          
Session        2024 Nov 03 08:05 PM 2024 Nov 06 12:54 PM 2024 Nov 10 02:56 PM   
Club                                                                            
4 Iron                          NaN                  NaN                  NaN   
5 Iron                          NaN                  NaN                  NaN   
5 Wood                          NaN                  NaN             3.916667   
6 Iron                          NaN                  NaN             9.941667   
7 Iron                     4.266667                  NaN             9.758571   
8 Iron                          NaN                  NaN            10.294118   
9 Iron                          NaN            10.725000                  NaN   
Driver                          NaN                  NaN             2.433333   
Gap Wedge                       NaN                  NaN             9.210000   
Lob Wedge                       NaN             3.733333                  NaN   
Pitching Wedge                  NaN                  NaN                  NaN   
Sand Wedge                      NaN                  NaN                  NaN   

                                                                               \
Golfer                                              Dave                        
Session        2024 Nov 29 07:05 PM 2024 Dec 02 08:45 PM 2024 Dec 08 03:19 PM   
Club                                                                            
4 Iron                     4.410526                  NaN                  NaN   
5 Iron                     4.936364                  NaN                  NaN   
5 Wood                          NaN                  NaN                  NaN   
6 Iron                     4.355556                  NaN                  NaN   
7 Iron                     6.966667                  NaN             4.709524   
8 Iron                          NaN                  NaN                  NaN   
9 Iron                     6.692308             6.271429             6.227273   
Driver                          NaN                  NaN                  NaN   
Gap Wedge                       NaN                  NaN                  NaN   
Lob Wedge                       NaN                  NaN                  NaN   
Pitching Wedge                  NaN                  NaN                  NaN   
Sand Wedge                 6.825000                  NaN                  NaN   

                                     ...            Total_yds  \
Golfer                               ...                 Dave   
Session    

In [93]:
grouped_df = df.groupby(['Session', 'Golfer', 'Club'])[num_columns].mean().reset_index()

# Convert Session column to datetime for proper sorting
grouped_df['Session'] = pd.to_datetime(grouped_df['Session'], format='%Y %b %d %I:%M %p')

# Pivot the table to have Clubs as rows and Sessions/Golfers as columns
result_df = grouped_df.pivot(index='Club', columns=['Session', 'Golfer'], values=num_columns)

# Replace missing values with NaN
result_df = result_df.fillna(np.nan)

# Extract subset for a specific golfer and remove multiindex on columns
# Ensure sessions are sorted by date
def extract_golfer_sessions(result_df, golfer):
    # Select columns for the specific golfer
    subset_df = result_df.xs(golfer, level='Golfer', axis=1, drop_level=False)  # Keep both levels but isolate the golfer

    # Convert 'Session' level to datetime and ensure all tuples match the structure
    new_columns = []
    for col in subset_df.columns:
        try:
            new_columns.append((col[0], pd.to_datetime(col[1])))
        except Exception as e:
            raise ValueError(f"Error converting column {col} to datetime: {e}")
    
    # Create new MultiIndex with proper datetime sorting
    subset_df.columns = pd.MultiIndex.from_tuples(new_columns, names=subset_df.columns.names)
    
    # Sort columns by the 'Session' level
    subset_df = subset_df.sort_index(axis=1, level='Session')
    return subset_df

# Example usage
golfer_subset_df = extract_golfer_sessions(result_df, 'Dave')
golfer_subset_df

C:\Users\dmomb\AppData\Local\Temp\ipykernel_2300\1615922769.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = df.groupby(['Session', 'Golfer', 'Club'])[num_columns].mean().reset_index()


ValueError: Length of names must match number of levels in MultiIndex.

In [91]:
golfer_subset_df.columns

MultiIndex([(               'AOA', '2024-12-02 20:45:00'),
            (               'AOA', '2024-12-08 15:19:00'),
            (               'AOA', '2024-12-08 16:34:00'),
            (               'AOA', '2024-12-08 19:54:00'),
            (               'AOA', '2024-12-21 17:21:00'),
            (               'AOA', '2024-11-03 14:56:00'),
            (               'AOA', '2024-11-03 20:05:00'),
            (               'AOA', '2024-11-06 12:54:00'),
            (               'AOA', '2024-11-10 14:56:00'),
            (               'AOA', '2024-11-17 16:49:00'),
            ...
            ('Vertical_Impact_in', '2024-12-08 19:54:00'),
            ('Vertical_Impact_in', '2024-12-21 17:21:00'),
            ('Vertical_Impact_in', '2024-11-03 14:56:00'),
            ('Vertical_Impact_in', '2024-11-03 20:05:00'),
            ('Vertical_Impact_in', '2024-11-06 12:54:00'),
            ('Vertical_Impact_in', '2024-11-10 14:56:00'),
            ('Vertical_Impact_in', '2024

In [82]:
carry_df = golfer_subset_df['Carry_yds'].sort_index(axis=1, level='Session')

In [84]:
carry_df

Session,2024-10-31 20:23:00,2024-11-03 14:56:00,2024-11-03 20:05:00,2024-11-06 12:54:00,2024-11-10 14:56:00,2024-11-17 10:57:00,2024-11-17 16:49:00,2024-11-29 19:05:00,2024-12-02 20:45:00,2024-12-08 15:19:00,2024-12-08 16:34:00,2024-12-08 19:54:00,2024-12-21 17:21:00
Club,,,,,,,,,,,,,
4 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.335333,NaN,NaN,NaN,NaN,NaN
5 Iron,NaN,NaN,NaN,NaN,NaN,1.218750,NaN,NaN,NaN,NaN,1.332000,NaN,1.302400
5 Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.335455,NaN,NaN,NaN,NaN,NaN
7 Iron,NaN,1.192857,1.1,NaN,NaN,1.222727,1.272778,1.287500,NaN,1.277619,1.267692,NaN,1.219130
8 Iron,NaN,NaN,NaN,NaN,1.142353,NaN,NaN,NaN,NaN,NaN,NaN,1.214545,NaN
9 Iron,NaN,NaN,NaN,NaN,1.118333,NaN,NaN,NaN,1.238571,1.145455,NaN,NaN,1.200714
Driver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gap Wedge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
import numpy as np
grouped_df = df.groupby(['Session', 'Golfer', 'Club'])[num_columns].mean().reset_index()

# Pivot the table to have Clubs as rows and Sessions/Golfers as columns
sessions_df = grouped_df.pivot(index='Club', columns=['Session', 'Golfer'], values=num_columns)

# Replace missing values with NaN
sessions_df = sessions_df.fillna(np.nan)

sessions_df

C:\Users\dmomb\AppData\Local\Temp\ipykernel_2300\1526759238.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = df.groupby(['Session', 'Golfer', 'Club'])[num_columns].mean().reset_index()


Ball_mph                                       \
Session        2024 Dec 02 08:45 PM                 2024 Dec 08 03:19 PM   
Golfer                         Alex       Dave Katy                 Alex   
Club                                                                       
4 Iron                          NaN        NaN  NaN                  NaN   
5 Iron                          NaN        NaN  NaN                  NaN   
5 Wood                          NaN        NaN  NaN                  NaN   
6 Iron                          NaN        NaN  NaN                  NaN   
7 Iron                          NaN        NaN  NaN                  NaN   
8 Iron                          NaN        NaN  NaN                  NaN   
9 Iron                          NaN  90.633333  NaN            99.128571   
Driver                          NaN        NaN  NaN                  NaN   
Gap Wedge                       NaN        NaN  NaN                  NaN   
Lob Wedge                       NaN        NaN  NaN                  NaN   
Pitching Wedge                  NaN        NaN  NaN                  NaN   
Sand Wedge                      NaN        NaN  NaN                  NaN   

                                                                      \
Session                        2024 Dec 08 04:34 PM                    
Golfer               Dave Katy                 Alex        Dave Katy   
Club                                                                   
4 Iron                NaN  NaN                  NaN         NaN  NaN   
5 Iron                NaN  NaN           121.073333  106.655000  NaN   
5 Wood                NaN  NaN                  NaN         NaN  NaN   
6 Iron                NaN  NaN                  NaN         NaN  NaN   
7 Iron          99.909524  NaN                  NaN   99.846154  NaN   
8 Iron                NaN  NaN                  NaN         NaN  NaN   
9 Iron          84.845455  NaN                  NaN         NaN  NaN   
Driver                NaN  NaN                  NaN         NaN  NaN   
Gap Wedge             NaN  NaN                  NaN         NaN  NaN   
Lob Wedge             NaN  NaN                  NaN         NaN  NaN   
Pitching Wedge        NaN  NaN                  NaN         NaN  NaN   
Sand Wedge            NaN  NaN                  NaN         NaN  NaN   

                                     ...   Vertical_Impact_in  \
Session        2024 Dec 08 07:54 PM  ... 2024 Nov 17 04:49 PM   
Golfer                         Alex  ...                 Katy   
Club                                 ...                        
4 Iron                          NaN  ...                  NaN   
5 Iron                          NaN  ...                  NaN   
5 Wood                          NaN  ...                  NaN   
6 Iron                          NaN  ...                  NaN   
7 Iron                          NaN  ...                  NaN   
8 Iron                      107.356  ...                  NaN   
9 Iron                          NaN  ...                  NaN   
Driver                          NaN  ...                  NaN   
Gap Wedge                       NaN  ...                  NaN   
Lob Wedge                       NaN  ...                  NaN   
Pitching Wedge                  NaN  ...                  NaN   
Sand Wedge                      NaN  ...                  NaN   

                                                                              \
Session        2024 Nov 17 10:57 AM           2024 Nov 29 07:05 PM             
Golfer                         Alex Dave Katy                 Alex Dave Katy   
Club                                                                           
4 Iron                          NaN  NaN  NaN                  NaN  NaN  NaN   
5 Iron                          NaN  NaN  NaN                  NaN  NaN  NaN   
5 Wood                          NaN  NaN  NaN                  NaN  NaN  NaN   
6 Iron                          NaN  NaN  NaN           

In [63]:
sessions_df.columns

MultiIndex([(          'Ball_mph', '2024 Dec 02 08:45 PM', 'Alex'),
            (          'Ball_mph', '2024 Dec 02 08:45 PM', 'Dave'),
            (          'Ball_mph', '2024 Dec 02 08:45 PM', 'Katy'),
            (          'Ball_mph', '2024 Dec 08 03:19 PM', 'Alex'),
            (          'Ball_mph', '2024 Dec 08 03:19 PM', 'Dave'),
            (          'Ball_mph', '2024 Dec 08 03:19 PM', 'Katy'),
            (          'Ball_mph', '2024 Dec 08 04:34 PM', 'Alex'),
            (          'Ball_mph', '2024 Dec 08 04:34 PM', 'Dave'),
            (          'Ball_mph', '2024 Dec 08 04:34 PM', 'Katy'),
            (          'Ball_mph', '2024 Dec 08 07:54 PM', 'Alex'),
            ...
            ('Vertical_Impact_in', '2024 Nov 17 04:49 PM', 'Katy'),
            ('Vertical_Impact_in', '2024 Nov 17 10:57 AM', 'Alex'),
            ('Vertical_Impact_in', '2024 Nov 17 10:57 AM', 'Dave'),
            ('Vertical_Impact_in', '2024 Nov 17 10:57 AM', 'Katy'),
            ('Vertical_Impact_in

In [61]:
# Extract subset for a specific golfer and remove multiindex on columns
def extract_golfer_sessions(df, golfer):
    subset_df = result_df.xs(golfer, level='Golfer', axis=1, drop_level=False)
    subset_df.columns = subset_df.columns.droplevel('Golfer')
    
    # Ensure the session order follows the original DataFrame
    session_order = grouped_df['Session'].unique()
    subset_df = subset_df.reindex(columns=session_order, level='Session')
    return subset_df

# Example usage
golfer_subset_df = extract_golfer_sessions(sessions_df, 'Dave')
golfer_subset_df

,2024 Dec 02 08:45 PM,2024 Dec 08 03:19 PM,2024 Dec 08 04:34 PM,2024 Dec 08 07:54 PM,2024 Dec 21 05:21 PM,2024 Nov 03 02:56 PM,2024 Nov 03 08:05 PM,2024 Nov 06 12:54 PM,2024 Nov 10 02:56 PM,2024 Nov 17 04:49 PM,2024 Nov 17 10:57 AM,2024 Nov 29 07:05 PM,2024 Oct 31 08:23 PM
Club,,,,,,,,,,,,,
4 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5 Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Driver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gap Wedge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
sessions_df

Ball_mph                                       \
Session        2024 Dec 02 08:45 PM                 2024 Dec 08 03:19 PM   
Golfer                         Alex       Dave Katy                 Alex   
Club                                                                       
4 Iron                          NaN        NaN  NaN                  NaN   
5 Iron                          NaN        NaN  NaN                  NaN   
5 Wood                          NaN        NaN  NaN                  NaN   
6 Iron                          NaN        NaN  NaN                  NaN   
7 Iron                          NaN        NaN  NaN                  NaN   
8 Iron                          NaN        NaN  NaN                  NaN   
9 Iron                          NaN  90.633333  NaN            99.128571   
Driver                          NaN        NaN  NaN                  NaN   
Gap Wedge                       NaN        NaN  NaN                  NaN   
Lob Wedge                       NaN        NaN  NaN                  NaN   
Pitching Wedge                  NaN        NaN  NaN                  NaN   
Sand Wedge                      NaN        NaN  NaN                  NaN   

                                                                      \
Session                        2024 Dec 08 04:34 PM                    
Golfer               Dave Katy                 Alex        Dave Katy   
Club                                                                   
4 Iron                NaN  NaN                  NaN         NaN  NaN   
5 Iron                NaN  NaN           121.073333  106.655000  NaN   
5 Wood                NaN  NaN                  NaN         NaN  NaN   
6 Iron                NaN  NaN                  NaN         NaN  NaN   
7 Iron          99.909524  NaN                  NaN   99.846154  NaN   
8 Iron                NaN  NaN                  NaN         NaN  NaN   
9 Iron          84.845455  NaN                  NaN         NaN  NaN   
Driver                NaN  NaN                  NaN         NaN  NaN   
Gap Wedge             NaN  NaN                  NaN         NaN  NaN   
Lob Wedge             NaN  NaN                  NaN         NaN  NaN   
Pitching Wedge        NaN  NaN                  NaN         NaN  NaN   
Sand Wedge            NaN  NaN                  NaN         NaN  NaN   

                                     ...   Vertical_Impact_in  \
Session        2024 Dec 08 07:54 PM  ... 2024 Nov 17 04:49 PM   
Golfer                         Alex  ...                 Katy   
Club                                 ...                        
4 Iron                          NaN  ...                  NaN   
5 Iron                          NaN  ...                  NaN   
5 Wood                          NaN  ...                  NaN   
6 Iron                          NaN  ...                  NaN   
7 Iron                          NaN  ...                  NaN   
8 Iron                      107.356  ...                  NaN   
9 Iron                          NaN  ...                  NaN   
Driver                          NaN  ...                  NaN   
Gap Wedge                       NaN  ...                  NaN   
Lob Wedge                       NaN  ...                  NaN   
Pitching Wedge                  NaN  ...                  NaN   
Sand Wedge                      NaN  ...                  NaN   

                                                                              \
Session        2024 Nov 17 10:57 AM           2024 Nov 29 07:05 PM             
Golfer                         Alex Dave Katy                 Alex Dave Katy   
Club                                                                           
4 Iron                          NaN  NaN  NaN                  NaN  NaN  NaN   
5 Iron                          NaN  NaN  NaN                  NaN  NaN  NaN   
5 Wood                          NaN  NaN  NaN                  NaN  NaN  NaN   
6 Iron                          NaN  NaN  NaN           

In [43]:
golfer_subset_df['Carry_yds']

Session,2024 Dec 02 08:45 PM,2024 Dec 08 03:19 PM,2024 Dec 08 04:34 PM,2024 Dec 08 07:54 PM,2024 Dec 21 05:21 PM,2024 Nov 03 02:56 PM,2024 Nov 03 08:05 PM,2024 Nov 06 12:54 PM,2024 Nov 10 02:56 PM,2024 Nov 17 04:49 PM,2024 Nov 17 10:57 AM,2024 Nov 29 07:05 PM,2024 Oct 31 08:23 PM
Club,,,,,,,,,,,,,
4 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.833333,NaN
5 Iron,NaN,NaN,139.920000,NaN,139.524000,NaN,NaN,NaN,NaN,NaN,121.850000,NaN,NaN
5 Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6 Iron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.490909,NaN
7 Iron,NaN,130.680952,133.330769,NaN,121.291304,120.872727,111.0,NaN,NaN,130.622222,124.718182,137.840000,NaN
8 Iron,NaN,NaN,NaN,108.272727,NaN,NaN,NaN,NaN,116.423529,NaN,NaN,NaN,NaN
9 Iron,117.166667,100.600000,NaN,NaN,110.278571,NaN,NaN,NaN,98.466667,NaN,NaN,NaN,NaN
Driver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gap Wedge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
import numpy as np
grouped_df = df.groupby(['Session', 'Golfer', 'Club'])[num_columns].mean().reset_index()

# Pivot the table to have Clubs as rows and Sessions/Golfers as columns
sessions_df = grouped_df.pivot(index='Club', columns=['Session', 'Golfer'], values=num_columns)

# Replace missing values with NaN
sessions_df = sessions_df.fillna(np.nan)

sessions_df

C:\Users\dmomb\AppData\Local\Temp\ipykernel_2300\1526759238.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df = df.groupby(['Session', 'Golfer', 'Club'])[num_columns].mean().reset_index()


Ball_mph                                       \
Session        2024 Dec 02 08:45 PM                 2024 Dec 08 03:19 PM   
Golfer                         Alex       Dave Katy                 Alex   
Club                                                                       
4 Iron                          NaN        NaN  NaN                  NaN   
5 Iron                          NaN        NaN  NaN                  NaN   
5 Wood                          NaN        NaN  NaN                  NaN   
6 Iron                          NaN        NaN  NaN                  NaN   
7 Iron                          NaN        NaN  NaN                  NaN   
8 Iron                          NaN        NaN  NaN                  NaN   
9 Iron                          NaN  90.633333  NaN            99.128571   
Driver                          NaN        NaN  NaN                  NaN   
Gap Wedge                       NaN        NaN  NaN                  NaN   
Lob Wedge                       NaN        NaN  NaN                  NaN   
Pitching Wedge                  NaN        NaN  NaN                  NaN   
Sand Wedge                      NaN        NaN  NaN                  NaN   

                                                                      \
Session                        2024 Dec 08 04:34 PM                    
Golfer               Dave Katy                 Alex        Dave Katy   
Club                                                                   
4 Iron                NaN  NaN                  NaN         NaN  NaN   
5 Iron                NaN  NaN           121.073333  106.655000  NaN   
5 Wood                NaN  NaN                  NaN         NaN  NaN   
6 Iron                NaN  NaN                  NaN         NaN  NaN   
7 Iron          99.909524  NaN                  NaN   99.846154  NaN   
8 Iron                NaN  NaN                  NaN         NaN  NaN   
9 Iron          84.845455  NaN                  NaN         NaN  NaN   
Driver                NaN  NaN                  NaN         NaN  NaN   
Gap Wedge             NaN  NaN                  NaN         NaN  NaN   
Lob Wedge             NaN  NaN                  NaN         NaN  NaN   
Pitching Wedge        NaN  NaN                  NaN         NaN  NaN   
Sand Wedge            NaN  NaN                  NaN         NaN  NaN   

                                     ...   Vertical_Impact_in  \
Session        2024 Dec 08 07:54 PM  ... 2024 Nov 17 04:49 PM   
Golfer                         Alex  ...                 Katy   
Club                                 ...                        
4 Iron                          NaN  ...                  NaN   
5 Iron                          NaN  ...                  NaN   
5 Wood                          NaN  ...                  NaN   
6 Iron                          NaN  ...                  NaN   
7 Iron                          NaN  ...                  NaN   
8 Iron                      107.356  ...                  NaN   
9 Iron                          NaN  ...                  NaN   
Driver                          NaN  ...                  NaN   
Gap Wedge                       NaN  ...                  NaN   
Lob Wedge                       NaN  ...                  NaN   
Pitching Wedge                  NaN  ...                  NaN   
Sand Wedge                      NaN  ...                  NaN   

                                                                              \
Session        2024 Nov 17 10:57 AM           2024 Nov 29 07:05 PM             
Golfer                         Alex Dave Katy                 Alex Dave Katy   
Club                                                                           
4 Iron                          NaN  NaN  NaN                  0.0  0.0  NaN   
5 Iron                          NaN  0.0  NaN                  0.0  NaN  NaN   
5 Wood                          NaN  NaN  NaN                  NaN  NaN  NaN   
6 Iron                          NaN  NaN  NaN           

In [39]:
grouped_df = df.groupby(['Golfer', 'Club'])[num_columns].mean().reset_index()

# Pivot the table to have Clubs as rows and Sessions/Golfers as columns
clubs_df = grouped_df.pivot(index='Club', columns=['Golfer'], values=num_columns)

# Replace missing values with NaN
clubs_df = clubs_df.fillna(np.nan)

clubs_df


Ball_mph                           Club_mph             \
Golfer                Alex        Dave       Katy        Alex       Dave   
Club                                                                       
4 Iron          125.331579  107.846667        NaN   90.021053  80.540000   
5 Iron          122.280769  105.541509        NaN   87.003846  81.264151   
5 Wood          125.891667         NaN        NaN  100.941667        NaN   
6 Iron          120.057143  104.518182        NaN   87.590476  78.390909   
7 Iron          108.238679   98.350388  72.440000   84.544340  78.951562   
8 Iron          108.942857   91.196429        NaN   85.595238  77.946429   
9 Iron          100.670968   87.807576  53.366667   80.774194  73.445455   
Driver          146.111111         NaN        NaN  108.677778        NaN   
Gap Wedge        81.500000         NaN        NaN   78.780000        NaN   
Lob Wedge       106.100000         NaN        NaN   81.400000        NaN   
Pitching Wedge         NaN   86.225641        NaN         NaN  71.294872   
Sand Wedge       67.825000         NaN        NaN   62.075000        NaN   

                          Smash_Factor                       Carry_yds  ...  \
Golfer               Katy         Alex      Dave      Katy        Alex  ...   
Club                                                                    ...   
4 Iron                NaN     1.391579  1.335333       NaN  178.847368  ...   
5 Iron                NaN     1.406538  1.300943       NaN  166.330769  ...   
5 Wood                NaN     1.248333       NaN       NaN  166.225000  ...   
6 Iron                NaN     1.371429  1.335455       NaN  165.685714  ...   
7 Iron          57.400000     1.281321  1.246953  1.280000  145.546226  ...   
8 Iron                NaN     1.273571  1.170714       NaN  150.719048  ...   
9 Iron          52.266667     1.246452  1.196061  1.008333  134.325806  ...   
Driver                NaN     1.345556       NaN       NaN  213.100000  ...   
Gap Wedge             NaN     1.033000       NaN       NaN   99.750000  ...   
Lob Wedge             NaN     1.306667       NaN       NaN  149.100000  ...   
Pitching Wedge        NaN          NaN  1.209231       NaN         NaN  ...   
Sand Wedge            NaN     1.090000       NaN       NaN   76.825000  ...   

                 DescentV Curve_Dist_yds           Lateral_Impact_in  \
Golfer               Katy           Alex Dave Katy              Alex   
Club                                                                   
4 Iron                NaN            NaN  NaN  NaN          0.000000   
5 Iron                NaN            NaN  NaN  NaN          0.234000   
5 Wood                NaN            NaN  NaN  NaN          0.000000   
6 Iron                NaN            NaN  NaN  NaN          0.000000   
7 Iron          27.920000            NaN  NaN  NaN          0.000000   
8 Iron                NaN            NaN  NaN  NaN          0.231905   
9 Iron          29.983333            NaN  NaN  NaN          0.262000   
Driver                NaN            NaN  NaN  NaN          0.000000   
Gap Wedge             NaN            NaN  NaN  NaN          0.000000   
Lob Wedge             NaN            NaN  NaN  NaN          0.000000   
Pitching Wedge        NaN            NaN  NaN  NaN               NaN   
Sand Wedge            NaN            NaN  NaN  NaN          0.000000   

                              Vertical_Impact_in                 
Golfer              Dave Katy               Alex      Dave Katy  
Club                                                             
4 Iron          0.000000  NaN           0.000000  0.000000  NaN  
5 Iron          0.370980  NaN           0.195200  0.218824  NaN  
5 Wood               NaN  NaN           0.000000       NaN  NaN  
6 Iron          0.000000  NaN           0.000000  0.000000  NaN  
7 Iron          0.128393  0.0           0.000000  0.109821  0.0  
8 Iron          0.239643  NaN           0.223571  0.087500  NaN  
9 Iron          

In [37]:
# Extract subset for a specific golfer and remove multiindex on columns
def extract_golfer_sessions(df, golfer):
    subset_df = df.xs(golfer, level='Golfer', axis=1, drop_level=False)
    subset_df.columns = subset_df.columns.droplevel('Golfer')
    return subset_df

# Example usage
golfer_subset_df = extract_golfer_sessions(sessions_df, 'Dave')
golfer_subset_df

,Ball_mph,Club_mph,Smash_Factor,Carry_yds,Total_yds,Roll_yds,Swing_H,Height_ft,Time_s,AOA,...,FTP,FTT,Dynamic_Loft,Club_Path,Launch_H,Launch_V,DescentV,Curve_Dist_yds,Lateral_Impact_in,Vertical_Impact_in
Club,,,,,,,,,,,,,,,,,,,,,
4 Iron,107.846667,80.540000,1.335333,133.833333,151.213333,17.400000,6.873333,34.420000,3.880000,4.780000,...,-5.220000,-1.000000,14.033333,4.213333,0.153333,10.246667,23.640000,NaN,0.000000,0.000000
5 Iron,105.541509,81.264151,1.300943,137.005660,147.950943,10.954717,7.259615,41.039623,4.266038,5.451923,...,-5.330769,-1.363462,16.786538,3.967308,-0.258491,11.915094,28.281132,NaN,0.370980,0.218824
5 Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6 Iron,104.518182,78.390909,1.335455,135.490909,144.263636,8.754545,7.645455,47.000000,4.427273,4.809091,...,-5.363636,-0.700000,17.909091,4.645455,0.418182,12.927273,30.954545,NaN,0.000000,0.000000
7 Iron,98.350388,78.951562,1.246953,128.041860,134.157364,6.113953,6.900781,51.082171,4.532558,5.405469,...,-3.635937,0.259375,23.715625,3.893750,1.306977,16.064341,34.652713,NaN,0.128393,0.109821
8 Iron,91.196429,77.946429,1.170714,113.221429,118.335714,5.128571,11.132143,49.817857,4.310714,7.903571,...,-8.350000,-1.325000,28.992857,7.039286,1.185714,17.750000,35.389286,NaN,0.239643,0.087500
9 Iron,87.807576,73.445455,1.196061,109.783333,114.356061,4.589394,4.703030,50.527273,4.325758,5.795455,...,-4.889394,-3.231818,30.656061,1.650000,-1.721212,19.716667,36.862121,NaN,0.424545,0.250455
Driver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gap Wedge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
sessions_df.columns

MultiIndex([(          'Ball_mph', 'Alex'),
            (          'Ball_mph', 'Dave'),
            (          'Ball_mph', 'Katy'),
            (          'Club_mph', 'Alex'),
            (          'Club_mph', 'Dave'),
            (          'Club_mph', 'Katy'),
            (      'Smash_Factor', 'Alex'),
            (      'Smash_Factor', 'Dave'),
            (      'Smash_Factor', 'Katy'),
            (         'Carry_yds', 'Alex'),
            (         'Carry_yds', 'Dave'),
            (         'Carry_yds', 'Katy'),
            (         'Total_yds', 'Alex'),
            (         'Total_yds', 'Dave'),
            (         'Total_yds', 'Katy'),
            (          'Roll_yds', 'Alex'),
            (          'Roll_yds', 'Dave'),
            (          'Roll_yds', 'Katy'),
            (           'Swing_H', 'Alex'),
            (           'Swing_H', 'Dave'),
            (           'Swing_H', 'Katy'),
            (         'Height_ft', 'Alex'),
            (         'Height_ft